In [18]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

characters = []

alphabet = 'abcdefghijklmnopqrstuvwxyz'.upper()
games = ['WC1','WC2','WC2X','WC3','WC3X','WoW','WoWBC','WoWWotLK','WoWC','WoWMoP','WoWWoD','WoWL','WoWBfA','WoWS','WoWD']

In [19]:
# get character name of all major characters

url = "https://wowpedia.fandom.com/wiki/Major_characters"
html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")

for s in soup.find_all('p'):
    text = s.text
    if "—" in text:
        
        d = {}
        
        t = text.replace("\n", "").split("—")

        name = t[0].strip().replace(" ", "_") #.replace(" ", "")

        expansions = t[1].replace(" ", "").split(",")
        expansions = [1 if i in expansions else 0 for i in games]
        
        if 1 in expansions:
            d['Name'] = name
            for i,g in enumerate(games):
                d[g] = expansions[i]
                
            characters.append(d)

In [20]:
# get lore characters

c_list = []

for letter in tqdm(alphabet):
    url = "https://wowpedia.fandom.com/wiki/Category:Lore_characters?pagefrom={}".format(letter)
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")
    
    for i in soup.find_all('li'):
        text = i.find('a')['href']
        verboten = ['/wiki/Genealogy', '/wiki/Local_Sitemap']
        if text[0:5] == '/wiki' and ":" not in text and text not in verboten:
            c_list.append(text.replace("%27", "'").replace("%22", '"').split("/wiki/")[1])

c_list = list(set(c_list))

for name in c_list:
    d = {}
    d['Name'] = name
    for i,g in enumerate(games):
        d[g] = 0 
        
    characters.append(d)

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:18<00:00,  1.43it/s]


In [22]:
df = pd.DataFrame(characters)
df

,Name,WC1,WC2,WC2X,WC3,WC3X,WoW,WoWBC,WoWWotLK,WoWC,WoWMoP,WoWWoD,WoWL,WoWBfA,WoWS,WoWD
0,Terokk,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,Talon_King_Ikiss,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,Archimonde,0,0,0,1,0,0,1,1,0,0,1,1,0,0,0
3,Kil'jaeden,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0
4,Mannoroth,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1966,Ruumbo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1967,Makkari,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1968,Ranger_Salissa,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1969,Prophet_Kheva,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
df.to_csv("df.csv", index=False)